suppose at each period, the underlying asset would move up to 1+u or it would go down to 1+d
so the price of the underlying is given by : 

\begin{align}
S_{n+1} = 
\left\{
\begin{array}{l}
u*S_{n} \\
d*S_{n} 
\end{array}
\right.
\end{align}


the probabiliy at wich the S goes up noted $\pi$ is : 
$\pi$ = $\frac{(1+r)^\Delta t - d}{u-d}$

# For 1 period

In [1]:
#Markdown to explain the process

In [2]:
class CCR_1P() : 
    def __init__(self, So, k, u, d, r, dt):
        self.So = So
        self.k = k
        self.u = u
        self.d = d
        self.r = r
        self.dt = dt
        self._pi    = self.__probability()
        self._Callupper = self.__CallUpperValue()
        self._Calldown  = self.__CallDownValue()
        self._PutUpper = self.__PutUpperValue()
        self._Putdown  = self.__PutDownValue()
    def pi(self) : 
        return self.__probability()
    def __CallUpperValue(self) : 
        return max(0, self.u*self.So - self.k)
    def __CallDownValue(self) : 
        return max(0, self.d*self.So - self.k)
    def __PutUpperValue(self) : 
        return max(0, -self.u*self.So + self.k)
    def __PutDownValue(self) : 
        return max(0, -self.d*self.So + self.k)
    def __probability(self) : 
        return ((1 + self.r)**self.dt - self.d)/(self.u - self.d)
    
    def CallPrice(self) : 
        return (self._pi*self._Callupper + (1-self._pi)*self._Calldown)/((1+self.r)**self.dt)
    def PutPrice(self) :
        return (self._pi*self._PutUpper + (1-self._pi)*self._Putdown)/((1+self.r)**self.dt)

## Example : 

In [3]:
So = 100
k = 99.9
u = 1.01
d = 0.99
r = 0.01
dt = 1/12

ccr_1P = CCR_1P(So, k, u, d, r, dt)
callOption = ccr_1P.CallPrice()
putOption = ccr_1P.PutPrice()
print("The price of a europen call options is : " + str(callOption))
print("The price of a europen put options is : " + str(putOption))
print("Call - Put = " + str(callOption - putOption))
print("S - k/(1+r) = " + str(So-k/(1+r)**(dt)))

The price of a europen call options is : 0.5951309125140798
The price of a europen put options is : 0.4123287424469252
Call - Put = 0.1828021700671546
S - k/(1+r) = 0.1828021700671485


# For 2 period

In [4]:
class CCR_2P() : 
    def __init__(self, So, k, u, d, r, dt):
        self.So = So
        self.k = k
        self.u = u
        self.d = d
        self.r = r
        self.dt = dt
    def pi(self) : 
        return self.__probability()
    def __probability(self) : 
        return ((1 + self.r)**self.dt - self.d)/(self.u - self.d)
    def __CallUpperValue(self) : 
        return max(0, (self.u**2)*self.So - self.k)
    def __CallDownValue(self) : 
        return max(0, (self.d**2)*self.So - self.k)
    def __CallMiddleValue(self) : 
        return max(0, (self.d*self.u)*self.So - self.k)
    
    def __PutUpperValue(self) : 
        return max(0, -(self.u**2)*self.So + self.k)
    def __PutDownValue(self) : 
        return max(0, -(self.d**2)*self.So + self.k)
    def __PutMiddleValue(self) : 
        return max(0, -(self.d*self.u)*self.So + self.k)
    
    def CallPrice(self) : 
        pi = self.__probability()
        return ((1/(1+self.r)**(2*self.dt))*(pi**2)*self.__CallUpperValue() + 2*pi*(1-pi)*self.__CallMiddleValue() + ((1-pi)**2)*self.__CallDownValue())
    def PutPrice(self) :
        pi = self.__probability()
        return ((1/(1+self.r)**(2*self.dt))*(pi**2)*self.__PutUpperValue() + 2*pi*(1-pi)*self.__PutMiddleValue() + ((1-pi)**2)*self.__PutDownValue())    

## Example : 

In [5]:
So = 100
k = 99
u = 1.01
d = 0.99
r = 0.01
dt = 1/12

ccr_2P = CCR_2P(So, k, u, d, r, dt)

callOption = ccr_2P.CallPrice()
putOption = ccr_2P.PutPrice()

print("The price of a europen call options is : " + str(callOption))
print("The price of a europen put options is : " + str(putOption))
print("Call - Put = " + str(callOption - putOption))
print("S - k/(1+r) = " + str(So-k/(1+r)**(2*dt)))

The price of a europen call options is : 1.3726550757210638
The price of a europen put options is : 0.20814099371067052
Call - Put = 1.1645140820103932
S - k/(1+r) = 1.1640443968117467


# For several periods

In [6]:
from math import *
def C(k, n) : 
        return factorial(n)/(factorial(n-k)*factorial(k))

In [58]:
class CCR_NP() : 
    def __init__(self, So, k, u, d, r, N, dt):
        """
        N : is the period length
        """
        self.So = So
        self.k = k
        self.u = u
        self.d = d
        self.r = r
        self.N = N
        self.dt = dt
    def pi(self) :
        return self.__probability()
    def __probability(self) : 
        return ((1 + self.r)**self.dt - self.d)/(self.u - self.d)
    def __CallValue(self, j) : 
        return max(0, (self.u**j)*(self.d**(N-j))*self.So - self.k)
    
    def __PutValue(self, j) : 
        return max(0, -(self.u**j)*(self.d**(N-j))*self.So + self.k)
    
    def CallPrice(self) : 
        pi = self.__probability()
        c = 0
        for i in range(N+1) : 
            c = c + C(i,N)*(pi**i)*((1-pi)**(N-i))*self.__CallValue(i)
        return c/(1+self.r)**(self.N*self.dt)
    def PutPrice(self) :
        pi = self.__probability()
        p = 0
        for i in range(N+1) : 
            p = p + C(i,N)*(pi**i)*((1-pi)**(N-i))*self.__PutValue(i)
        return p/(1+self.r)**(self.N*self.dt)

## Example : 

In [61]:
So = 100
k = 99
u = 1.01
d = 0.99
r = 0.01
dt = 1/12
N = 100

ccr_NP = CCR_NP(So, k, u, d, r, N, dt)

callOption = ccr_NP.CallPrice()
putOption = ccr_NP.PutPrice()
print("The price of a europen call options is : " + str(callOption))
print("The price of a europen put options is : " + str(putOption))
print("Call - Put = " + str(callOption - putOption))
print("S - k/(1+r) = " + str(So-k/(1+r)**(N*dt)))

The price of a europen call options is : 9.78314478793337
The price of a europen put options is : 0.9052503420799677
Call - Put = 8.877894445853402
S - k/(1+r) = 8.877894445854139
